In [6]:
import os.path as osp
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [2]:
wv.similarity("Test", "Exam")

0.07620491

In [8]:
import pandas as pd
things_categories = pd.read_csv(osp.join("../", "things_concepts.tsv"), sep="\t")
things_categories.head()

,Word,uniqueID,Example image,Bigram,Percent_known,Rank (combining COCA/concreteness),Concreteness (M),COCA word freq (online),COCA word freq,COCA (dispersion),...,Top-down Category (WordNet),Top-down Category (manual selection),All Bottom-up Categories,WordNet Synonyms,WordNet ID,Wordnet ID2,Wordnet ID3,Wordnet ID4,"Definition (from WordNet, Google, or Wikipedia)",Number of word meanings in list
0,aardvark,aardvark,https://imgur.com/LAJGlN0,0,1.00,18158.5,4.68,53,28.0,0.78,...,animal,animal,animal,"aardvark, ant_bear, anteater, Orycteropus_afer",n02082791,aardvark%1:05:00::,aardvark#1,aardvark.n.01,nocturnal burrowing mammal of the grasslands o...,1
1,abacus,abacus,https://imgur.com/peZeM0l,0,0.93,17741.0,4.52,188,97.0,0.86,...,NaN,home decor,mathematical device,abacus,n02666196,abacus%1:06:00::,abacus#2,abacus.n.02,a calculator that performs arithmetic function...,1
2,accordion,accordion,https://imgur.com/GgGvdZR,0,0.97,8541.0,4.86,816,735.0,0.90,...,musical instrument,musical instrument,musical instrument,"accordion, piano_accordion, squeeze_box",n02672831,accordion%1:06:00::,accordion#1,accordion.n.01,a portable box-shaped free-reed instrument; th...,1
3,acorn,acorn,https://imgur.com/YfIB5lM,0,1.00,7723.5,4.96,1289,692.0,0.85,...,fruit,NaN,nut,acorn,n12267677,acorn%1:20:00::,acorn#1,acorn.n.01,fruit of the oak tree: a smooth thin-walled nu...,1
4,air conditioner,air_conditioner,https://imgur.com/KqYNwWH,1,1.00,12115.0,4.93,943,NaN,NaN,...,NaN,electronic device,home appliance,"air_conditioner, air_conditioning",n02686379,air_conditioner%1:06:00::,air_conditioner#1,air_conditioner.n.01,a system that keeps air cool and dry,1


In [38]:
things_categories.shape

(1854, 25)

In [52]:
key_categories = {
    "natural": ["natural", "animal", "plant"],
    "artificial": ["artificial", "machine"]
}

In [53]:
"air" in wv

True

In [60]:
# for r in things_categories.T.items():
for index, row in things_categories[18:20].iterrows():
    word = row['Word']
    sim_vals = dict()
    if word not in wv:
        print(word, "Out of Dict")
        continue
    for cat in key_categories:
        best_sim = -999
        best_term_in_cat = ""
        for term in key_categories[cat]:
            sim_val = wv.similarity(word, term)
            if sim_val > best_sim:
                best_sim = sim_val
                best_term_in_cat = term
        sim_vals[cat] = (best_sim, cat, best_term_in_cat)
    best_category = ""
    best_sim_val = -99
    for k in sim_vals:
        v = sim_vals[k][0]
        if v > best_sim_val:
            best_sim_val = v
            best_category = sim_vals[k]
    print(word, best_category, best_sim_val, sim_vals)
    

amber (0.21142913, 'natural', 'natural') 0.21142913 {'natural': (0.21142913, 'natural', 'natural'), 'artificial': (0.19946404, 'artificial', 'artificial')}
ambulance (0.17784567, 'natural', 'animal') 0.17784567 {'natural': (0.17784567, 'natural', 'animal'), 'artificial': (0.12225049, 'artificial', 'artificial')}


In [72]:
model_gigaword.similarity("artificial", "ambulance"), model_gigaword.similarity("animal", "ambulance")

(0.010503667, 0.14982235)

In [68]:
model_gigaword = api.load("glove-wiki-gigaword-300")


[==================================================] 100.0% 376.1/376.1MB downloaded


In [76]:
import pyperclip
pyperclip.copy('The text to be copied to the clipboard.')
spam = pyperclip.paste()

In [144]:
prompt = """For each of the words below, tell me which of the following categories it is most similar to: Artificial, Natural
Format your response as so
<word>, <category>
"""
start_idx = 18 # increment this
batch_size = 100
N = len(things_categories[start_idx * batch_size:(start_idx + 1) * batch_size])
prompt += f"""
There are {N} words shown below, make sure to give an answer for each of them.
The words are:

"""
for index, row in things_categories[start_idx * batch_size:(start_idx + 1) * batch_size].iterrows():
    word = row['Word']
    prompt += f"{word}, "
pyperclip.copy(prompt)

In [152]:
things_clustering = pd.read_csv("../things_clustering.csv")
things_clustering.tail()

,object,category
1710,triangle,Natural
1711,tricycle,Artificial
1712,trident,Artificial
1713,trigger,Artificial
1714,tripod,Artificial


In [154]:
# you may need to fix a few entries due to the output missing a word or something, use the code below to find out whats missing

In [155]:
for i in range(len(things_clustering)):
    # verify the object name is right
    word = things_clustering.iloc[i]["object"]
    
    word2 = things_categories.iloc[i]["Word"]
    if word != word2:
        print("Mistake", i, "Ground Truth:", word2, "Got", word)

Mistake 87 Ground Truth: bat Got bat (flying mammal)
Mistake 88 Ground Truth: bat Got bat (sports equipment)
Mistake 92 Ground Truth: baton Got baton (musical instrument)
Mistake 93 Ground Truth: baton Got baton (relay race)
Mistake 94 Ground Truth: baton Got baton (police)
Mistake 95 Ground Truth: baton Got baton (majorette)
Mistake 174 Ground Truth: bow Got bow (archery)
Mistake 175 Ground Truth: bow Got bow (ribbon)
Mistake 176 Ground Truth: bow Got bow (knot)
Mistake 187 Ground Truth: bracelet Got bracelet (jewelry)
Mistake 188 Ground Truth: bracelet Got bracelet (orthopedic)


In [130]:
len(things_clustering)

700

In [ ]:
# Javascript code to copy all the chatgpt outputs
"""
function copyToClipboard(text) {
    var dummy = document.createElement("textarea");
    // to avoid breaking orgain page when copying more words
    // cant copy when adding below this code
    // dummy.style.display = 'none'
    document.body.appendChild(dummy);
    //Be careful if you use texarea. setAttribute('value', value), which works with "input" does not work with "textarea". – Eduard
    dummy.value = text;
    dummy.select();
    document.execCommand("copy");
    document.body.removeChild(dummy);
}
elements = document.getElementsByClassName("prose");
out = "object, category\n"
for (let i = 0; i < elements.length; i++) {
    let txt = elements[i].textContent;
    out += txt + "\n";
}
copyToClipboard(out);
"""